In [25]:
%pip install -r to_onnx.requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [26]:
import os
import boto3

import joblib
import skl2onnx
from skl2onnx.common.data_types import FloatTensorType

In [27]:
model = joblib.load("build/model.joblib")

In [28]:
initial_type = [('float_input', FloatTensorType([None, 2]))]
onx = skl2onnx.convert_sklearn(model, initial_types=initial_type)


MODEL_ID = os.getenv("BUILD_ID", "dev")
onnx_filename = f"build/model.{MODEL_ID}.onnx"

with open(onnx_filename, "wb") as f:
    f.write(onx.SerializeToString())

Test

In [29]:
import numpy as np
import onnxruntime as rt

sess = rt.InferenceSession(onnx_filename, providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

print(input_name)
print(output_name)

float_input
variable


In [30]:
student_input =  np.array([[4, 2]], dtype=np.float32)

sess.run([output_name], {input_name: student_input})[0]

array([[10.804643]], dtype=float32)

Upload

In [31]:
key_id = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region = os.getenv("AWS_DEFAULT_REGION")
endpoint = os.getenv("AWS_S3_ENDPOINT")
bucket_name = os.getenv("AWS_S3_BUCKET")

s3_client = boto3.client(
    "s3",
    region,
    aws_access_key_id=key_id,
    aws_secret_access_key=secret_key,
    endpoint_url=endpoint,
    use_ssl=True
)

s3_client.upload_file(onnx_filename, bucket_name, Key=onnx_filename)

print(f"File {onnx_filename} uploaded to S3!")

File build/model.dev.onnx uploaded to S3!


This ONNX model works with Triton runtime 23.05 Model Server and the `onnx - 1` framework.